<a href="https://colab.research.google.com/github/CleverEnd/Fundamentos-Ciencia-De-Datos-Modulo2-/blob/main/challenge_2DataModelingJosephTrejo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Challenge: Múltiples variables almacenadas en una columna.

Vamos a tratar ahora con el siguiente dataset: Registros de tuberculosis de la Organización Mundial de la Salud
Este conjunto de datos documenta el recuento de casos confirmados de tuberculosis por país, año, edad y sexo.

## Problemas:
- Algunas columnas contienen varios valores: sexo y edad.
- Mezcla de ceros y valores perdidos NaN. Esto se debe al proceso de recopilación de datos y la distinción es importante para este conjunto de datos.

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("tb-raw.csv")
df

country  year   m014  m1524   m2534  m3544  m4554  m5564    m65  mu   f014
0      AD  2000    0.0    0.0     1.0    0.0      0      0    0.0 NaN    NaN
1      AE  2000    2.0    4.0     4.0    6.0      5     12   10.0 NaN    3.0
2      AF  2000   52.0  228.0   183.0  149.0    129     94   80.0 NaN   93.0
3      AG  2000    0.0    0.0     0.0    0.0      0      0    1.0 NaN    1.0
4      AL  2000    2.0   19.0    21.0   14.0     24     19   16.0 NaN    3.0
5      AM  2000    2.0  152.0   130.0  131.0     63     26   21.0 NaN    1.0
6      AN  2000    0.0    0.0     1.0    2.0      0      0    0.0 NaN    0.0
7      AO  2000  186.0  999.0  1003.0  912.0    482    312  194.0 NaN  247.0
8      AR  2000   97.0  278.0   594.0  402.0    419    368  330.0 NaN  121.0
9      AS  2000    NaN    NaN     NaN    NaN      1      1    NaN NaN    NaN

Para ordenar este conjunto de datos, necesitamos eliminar los diferentes valores del encabezado y SEPARARLOS en filas.

### Instrucciones:
- Primero, necesitaremos fusionar las columnas de sexo + grupo de edad en una sola.
- Una vez que tengamos esa única columna, derivaremos tres columnas de ella: sexo, age_lower y age_upper. Con ellos, podremos construir correctamente un conjunto de datos ordenado.

In [4]:
# Utiliza melt como en primer challenge para realizar el melt sobre las columnas que correspondan
df = pd.melt(df, id_vars=["country","year"], value_name="frecuencia", var_name="group")
df

country  year group  frecuencia
0       AD  2000  m014         0.0
1       AE  2000  m014         2.0
2       AF  2000  m014        52.0
3       AG  2000  m014         0.0
4       AL  2000  m014         2.0
..     ...   ...   ...         ...
85      AM  2000  f014         1.0
86      AN  2000  f014         0.0
87      AO  2000  f014       247.0
88      AR  2000  f014       121.0
89      AS  2000  f014         NaN

[90 rows x 4 columns]

In [5]:
# Utiliza str.extract como en el primer challenge para separar el género y el rango de edad
genero = df['group'].str.extract(r'([a-z])(.*)([\d]{2})', expand=False) # wildcard
genero

0  1   2
0   m  0  14
1   m  0  14
2   m  0  14
3   m  0  14
4   m  0  14
.. .. ..  ..
85  f  0  14
86  f  0  14
87  f  0  14
88  f  0  14
89  f  0  14

[90 rows x 3 columns]

In [6]:
# Renombre las columnas como "sex", "age_lower", "age_upper" según corresponda
genero.columns = ['sex', 'age_lower', 'age_upper']
genero

sex age_lower age_upper
0    m         0        14
1    m         0        14
2    m         0        14
3    m         0        14
4    m         0        14
..  ..       ...       ...
85   f         0        14
86   f         0        14
87   f         0        14
88   f         0        14
89   f         0        14

[90 rows x 3 columns]

In [7]:
# Crea la columna `age` uniendo los valores de las columnas `age_lower` y `age_upper`
genero['age'] = genero['age_lower'] + '-' + genero['age_upper']
genero.drop(columns=['age_lower', 'age_upper'], inplace=True)
genero

sex   age
0    m  0-14
1    m  0-14
2    m  0-14
3    m  0-14
4    m  0-14
..  ..   ...
85   f  0-14
86   f  0-14
87   f  0-14
88   f  0-14
89   f  0-14

[90 rows x 2 columns]

In [8]:
# Concatenamos los data frames modificados
df.drop(columns='group', inplace=True)
final = pd.concat([df, genero], axis=1)
final.sort_values(['country', 'year'])

country  year  frecuencia  sex    age
0       AD  2000         0.0    m   0-14
10      AD  2000         0.0    m  15-24
20      AD  2000         1.0    m  25-34
30      AD  2000         0.0    m  35-44
40      AD  2000         0.0    m  45-54
..     ...   ...         ...  ...    ...
49      AS  2000         1.0    m  45-54
59      AS  2000         1.0    m  55-64
69      AS  2000         NaN    m    -65
79      AS  2000         NaN  NaN    NaN
89      AS  2000         NaN    f   0-14

[90 rows x 5 columns]

In [9]:
# Removemos columnas que ya no son necesarias
final['age'].unique()

array(['0-14', '15-24', '25-34', '35-44', '45-54', '55-64', '-65', nan],
      dtype=object)

In [10]:
final

country  year  frecuencia sex   age
0       AD  2000         0.0   m  0-14
1       AE  2000         2.0   m  0-14
2       AF  2000        52.0   m  0-14
3       AG  2000         0.0   m  0-14
4       AL  2000         2.0   m  0-14
..     ...   ...         ...  ..   ...
85      AM  2000         1.0   f  0-14
86      AN  2000         0.0   f  0-14
87      AO  2000       247.0   f  0-14
88      AR  2000       121.0   f  0-14
89      AS  2000         NaN   f  0-14

[90 rows x 5 columns]

## Resultado
Al finalizar, deberías obtener un dataframe similar a este:

![image.png](https://storage.googleapis.com/campus-cvs/lectures/tidyDataChallenge2.PNG)